<div style="position: relative;">
<img src="https://user-images.githubusercontent.com/7065401/98728503-5ab82f80-2378-11eb-9c79-adeb308fc647.png"></img>

<h1 style="color: white; position: absolute; top:30%; left:10%;">
    PostgreSQL for Python Developers
</h1>

<h3 style="color: #ef7d22; font-weight: normal; position: absolute; top:48%; left:10%;">
    David Mertz, Ph.D.
</h3>

<h3 style="color: #ef7d22; font-weight: normal; position: absolute; top:55%; left:10%;">
    Data Scientist
</h3>
</div>
<br/>

# Data types in PostgreSQL

Columns in PostgreSQL may utilize a wide range of data types.  Many of these are features in SQL standards.  Others are PostgreSQL custom data types with particular functions and syntax provided to work with them.  Still others—such as support for XML and JSON—are supported in the most recent SQL standards, but were added in somewhat different forms earlier and independently within PostgreSQL.

In [1]:
import psycopg2
cred = dict(user='ine_student', password='ine-password', database='ine', host='localhost')
conn = psycopg2.connect(**cred)
cur = conn.cursor()

## Numeric types

PostgreSQL numeric types consist of two, four, and eight byte integers, four and eight byte floating-point numbers, and selectable-precision decimals. 

In [2]:
sql = """
CREATE TABLE numbers (
    a SMALLINT DEFAULT NULL,        -- -32768 to +32767
    b INTEGER DEFAULT NULL,         -- approx -2e9 to +2e9
    c BIGINT DEFAULT NULL,          -- approx -9e18 to +9e18  
    d DECIMAL(10,9) DEFAULT NULL,   -- up to 16,383 decimals digits!
    e DECIMAL(40,25) DEFAULT NULL,  -- up to 131,072 leading digits!
    f REAL DEFAULT NULL,            -- 32-bit float
    g DOUBLE PRECISION DEFAULT NULL,-- 64-bit float
    h MONEY DEFAULT NULL            -- Monetary (locale dependent)
    );
"""
cur.execute('DROP TABLE IF EXISTS numbers;')
cur.execute(sql)
conn.commit()

In [3]:
pi = '3.14159265358979323846264338327950288419716939937510582097494459230781640628620899862803482534'
for col in 'abcdefgh':
    try:
        cur.execute(f"INSERT INTO numbers ({col}) VALUES ({pi});")
    except Exception as err:
        print(f"Column {col}: {str(err).strip()}")
    else:
        cur.execute(f'SELECT {col} FROM numbers;')
        print(f"Column {col}: {cur.fetchall()[0]}")
    finally:
        conn.rollback()

Column a: (3,)
Column b: (3,)
Column c: (3,)
Column d: (Decimal('3.141592654'),)
Column e: (Decimal('3.1415926535897932384626434'),)
Column f: (3.1415927,)
Column g: (3.141592653589793,)
Column h: ('$3.14',)


In [4]:
for col in 'abcdefgh':
    try:
        cur.execute(f"INSERT INTO numbers ({col}) VALUES (1000000000000);")
    except Exception as err:
        print(f"Column {col}: {str(err).strip()}")
    else:
        cur.execute(f'SELECT {col} FROM numbers;')
        print(f"Column {col}: {cur.fetchall()[0]}")
    finally:
        conn.rollback()

Column a: smallint out of range
Column b: integer out of range
Column c: (1000000000000,)
Column d: numeric field overflow
DETAIL:  A field with precision 10, scale 9 must round to an absolute value less than 10^1.
Column e: (Decimal('1000000000000.0000000000000000000000000'),)
Column f: (1000000000000.0,)
Column g: (1000000000000.0,)
Column h: ('$1,000,000,000,000.00',)


## Character data

There are two types of character data in PostgreSQL, fixed length which will be space padded, and varying length.  For varying length, you may limit to a maximum length.  In some other database systems, using fixed length character data can be more efficient; however, in PostgreSQL, there is not disadvantage to varying length (possibly slight performance gain), and it should almost always be used in new data tables.

In [5]:
sql = """
CREATE TABLE characters (
    a CHAR(5),        
    b CHAR(50),
    c VARCHAR(5),  -- synonym for CHARACTER VARYING
    d VARCHAR(50),
    e TEXT         -- synonym for VARCHAR with no length
    );
"""
cur.execute('DROP TABLE IF EXISTS characters;')
cur.execute(sql)
conn.commit()

In [6]:
s = 'There is no performance difference'
for col in 'abcde':
    try:
        cur.execute(f"INSERT INTO characters ({col}) VALUES ('{s}');")
    except Exception as err:
        print(f"Column {col}: {str(err).strip()}")
    else:
        cur.execute(f'SELECT {col} FROM characters;')
        print(f"Column {col}: {cur.fetchall()[0]}")
    finally:
        conn.rollback()

Column a: value too long for type character(5)
Column b: ('There is no performance difference                ',)
Column c: value too long for type character varying(5)
Column d: ('There is no performance difference',)
Column e: ('There is no performance difference',)


## Binary data

In many database systems, and in the SQL standard, the type BLOB is used.  In PostgreSQL, the equivalent is BYTEA.

In [7]:
from pickle import dumps, loads
cur.execute("DROP TABLE IF EXISTS pickles;")
cur.execute("CREATE TABLE pickles (name TEXT, bytes BYTEA);")
data = [('tuple', dumps(("a string", 1+2j))), ('dict', dumps({'this': 4, 'that': 1.23}))]
cur.executemany("INSERT INTO pickles VALUES (%s, %s)", data)

In [8]:
cur.execute("SELECT * FROM pickles;")
for row in cur:
    print(row[0])
    print(bytes(row[1]))
    print(loads(row[1]))
    print()

tuple
b'\x80\x04\x95;\x00\x00\x00\x00\x00\x00\x00\x8c\x08a string\x94\x8c\x08builtins\x94\x8c\x07complex\x94\x93\x94G?\xf0\x00\x00\x00\x00\x00\x00G@\x00\x00\x00\x00\x00\x00\x00\x86\x94R\x94\x86\x94.'
('a string', (1+2j))

dict
b'\x80\x04\x95\x1e\x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\x04this\x94K\x04\x8c\x04that\x94G?\xf3\xae\x14z\xe1G\xaeu.'
{'this': 4, 'that': 1.23}



## Date and time

We saw in earlier lessons examples of timestamps, with and without timezone information.  Separate DATE, TIME, and INTERVAL types are also available.  An interval is simply a `timedelta` in Python.  All types with time component have microsecond resolution.

If you work with time data of higher resolution, such as Pandas' `datetime64[ns]`, you will need to use a custom storage style, such as strings of sufficient lengths, or numeric types that you cast within your application.

Date inputs may take a variety of string formats that are heuristically analyzed.  Where ambiguity may exist, modes of MDY,  DMY, and YMD may be chosen with the SET command.

In [9]:
cur.execute("DROP TABLE IF EXISTS dates;")
cur.execute("CREATE TABLE dates (date DATE);")

examples = [('2020-12-25',), ('Dec 25, 2020',), ('12/25/20',)]
cur.executemany("INSERT INTO dates VALUES (%s)", examples)
cur.execute("SELECT * FROM dates;")
cur.fetchall()

[(datetime.date(2020, 12, 25),),
 (datetime.date(2020, 12, 25),),
 (datetime.date(2020, 12, 25),)]

## Booleans and enumerations

A variety of strings can be used to indicate Boolean values.  From Python it is largely moot since you will normally is interpolation of a Python Boolean.  But at times you may encounter strings and wish to use them directly.

You can also create enumerated types with a finite list of permitted values.

In [10]:
cur.execute("DROP TABLE IF EXISTS truth;")
cur.execute("CREATE TABLE truth (bool BOOLEAN);")
vals = [(v,) for v in 
        (True, False, None,
         'TRUE', 't', 'true', 'y', 'yes', 'on', '1',
        'FALSE', 'f', 'false', 'n', 'no', 'off', '0')]

cur.executemany("INSERT INTO truth VALUES (%s);", vals)
cur.execute("SELECT * FROM truth;")
print([v[0] for v in cur.fetchall()])
    

[True, False, None, True, True, True, True, True, True, True, False, False, False, False, False, False, False]


In [11]:
cur.execute("DROP TABLE IF EXISTS categorical;")
cur.execute("DROP TYPE IF EXISTS mood;")
cur.execute("CREATE TYPE mood AS ENUM ('sad', 'ok', 'happy');")
cur.execute("CREATE TABLE categorical (day SMALLINT, feeling MOOD);")

for n, feels in enumerate(['happy', 'sad', 'ok', 'uncertain']):
    try:
        cur.execute("INSERT INTO categorical VALUES (%s, %s);", (n, feels))
    except Exception as err:
        print(f"Values ({n}, {feels}): {str(err).strip()}")
        conn.rollback()
    else:
        conn.commit()

cur.execute(f'SELECT * FROM categorical;')
cur.fetchall()

Values (3, uncertain): invalid input value for enum mood: "uncertain"
LINE 1: INSERT INTO categorical VALUES (3, 'uncertain');
                                           ^


[(0, 'happy'), (1, 'sad'), (2, 'ok')]

## Geometric types

PostgreSQL offers a wide range of geometric types.  All of them describe some particular subset of a Cartesian plain.  The type POINT was used in prior lessons.  We also have: 

| Type    | Size         | Description         | Representation
|---------|--------------|---------------------|-------------------
| POINT   | 16 bytes     | Point on a plane    | (x,y)
| LINE    | 32 bytes     | Infinite line       | {A,B,C} `Ax+By+C=0`
| LSEG    | 32 bytes     | Finite line segment | [(x1,y1),(x2,y2)]
| BOX     | 32 bytes     | Rectangular box     | (x1,y1),(x2,y2)
| PATH    | 16+16n bytes | Closed; open        | ((x1,y1),...); [(x1,y1),...]
| POLYGON | 40+16n bytes | Polygon             | ((x1,y1),...)
| CIRCLE  | 24 bytes     | Circle              | <(x,y),r> `<(center), radius>`


The types alone are useful, but operations upon them are what is powerful (more details at https://www.postgresql.org/docs/current/functions-geometry.html).

In [12]:
from collections import namedtuple
Shapes = namedtuple("Shapes", "point lseg box circle")
cur.execute("DROP TABLE IF EXISTS shapes;")
cur.execute("""
    CREATE TABLE shapes (
        pnt POINT,
        seg LSEG,
        box BOX,
        crc CIRCLE
    );""")

In [13]:
data = [("(1, 1)", "[(1, 1), (2, 3)]", "(1, 1), (2, 3)", "<(1, 1), 1>"),
        ("(2, 2)", "[(2, 3), (1, 1)]", "(2, 3), (1, 1)", "<(4, 4), 1>"),
        ("(3, 3)", "[(-1, -3), (2, -4)]", "(1, 3), (2, 4)", "<(1, 1), 1.5>")
       ]
cur.executemany("INSERT INTO shapes VALUES (%s, %s, %s, %s);", data)

We can ask various questions of these shapes.

In [14]:
cur.execute("SELECT * FROM shapes;")
for row in cur:
    print(Shapes(*row))

Shapes(point='(1,1)', lseg='[(1,1),(2,3)]', box='(2,3),(1,1)', circle='<(1,1),1>')
Shapes(point='(2,2)', lseg='[(2,3),(1,1)]', box='(2,3),(1,1)', circle='<(4,4),1>')
Shapes(point='(3,3)', lseg='[(-1,-3),(2,-4)]', box='(2,4),(1,3)', circle='<(1,1),1.5>')


In [15]:
# Equal area
cur.execute("SELECT * FROM shapes WHERE box = BOX '(0,0),(2,1)';")
for row in cur:
    print(Shapes(*row))

Shapes(point='(1,1)', lseg='[(1,1),(2,3)]', box='(2,3),(1,1)', circle='<(1,1),1>')
Shapes(point='(2,2)', lseg='[(2,3),(1,1)]', box='(2,3),(1,1)', circle='<(4,4),1>')


In [16]:
# Equal object
cur.execute("SELECT * FROM shapes WHERE crc ~= CIRCLE '<(1,1), 1>';")
for row in cur:
    print(Shapes(*row))

Shapes(point='(1,1)', lseg='[(1,1),(2,3)]', box='(2,3),(1,1)', circle='<(1,1),1>')


In [17]:
# Intersects
cur.execute("SELECT * FROM shapes WHERE LSEG '[(0.1, 0.1), (5, 4)]' ?# box;")
for row in cur:
    print(Shapes(*row))

Shapes(point='(1,1)', lseg='[(1,1),(2,3)]', box='(2,3),(1,1)', circle='<(1,1),1>')
Shapes(point='(2,2)', lseg='[(2,3),(1,1)]', box='(2,3),(1,1)', circle='<(4,4),1>')


In [18]:
# Intersection of two objects in columns
cur.execute("SELECT * FROM shapes WHERE seg ?# box;")
for row in cur:
    print(Shapes(*row))

Shapes(point='(1,1)', lseg='[(1,1),(2,3)]', box='(2,3),(1,1)', circle='<(1,1),1>')
Shapes(point='(2,2)', lseg='[(2,3),(1,1)]', box='(2,3),(1,1)', circle='<(4,4),1>')


In [19]:
# Contained in
cur.execute("SELECT * FROM shapes WHERE pnt <@ crc;")
for row in cur:
    print(Shapes(*row))

Shapes(point='(1,1)', lseg='[(1,1),(2,3)]', box='(2,3),(1,1)', circle='<(1,1),1>')


## Text search types

One area where PostgreSQL is very powerful is in performing efficient full-text search.  For that, we can create some custom types to enable this.  For this example, let us put the Project Gutenberg book, _Introduction to the study of the history of language_, by Herbert Augustus Strong and Willem Sijbrand Logeman and Benjamin Ide Wheeler (1891) into a PostgreSQL table, in a special way.

In [20]:
# Create one text for each paragraph
paras = open('../data/58650-0.txt').read().split('\n\n')
len(paras)

3247

The format used here will allow many books to be loaded.  For the example, only one is used.

In [21]:
sql_books = """
CREATE TABLE books (
    book_id TEXT,
    para_num INTEGER,
    vector TSVECTOR,
    raw_text TEXT
);
"""
cur.execute("DROP TABLE IF EXISTS books;")
cur.execute(sql_books)
conn.commit()

In [22]:
for n, para in enumerate(paras):
    # Add both the vector and full text to row
    sql = "INSERT INTO books VALUES (%s, %s, to_tsvector(%s), %s);"
    cur.execute(sql, ('58650-0.txt', n, para, para))
conn.commit()

Looking at what is added, we see that a TSVECTOR both creates an abstraction of words and also stems them.  Picking a paragraph from somewhere in the middle, we see, for example, the stem `applic` being indexed. Other words are noted with numbers to indicate the several positions where they occur in the text.

In [23]:
cur.execute("SELECT vector FROM books WHERE book_id = '58650-0.txt' AND para_num = 181;")
cur.fetchone()

("'applic':13 'bear':112 'brisban':154 'call':148 'cf':152 'charl':35,64,82,96,101 'circl':78 'circumst':87 'close':94 'come':17 'concret':8 'defin':91,123 'definit':7,160 'denot':19 'differ':15 'employ':164 'england':105 'even':59,130 'famili':73 'first':103 'franc':100 'general':3 'happen':139 'identifi':40,67 'import':144 'inde':43 'known':84 'larg':135 'le':170 'london':155 'man':33 'mani':110 'may':25 'melbourn':153 'mention':57 'mere':28 'must':89 'name':4,12,63,115,119,151 'natur':90 'nearer':159 'neighbourhood':128 'newton':169 'newton-le-willow':168 'one':23 'order':37 'otherwis':157 'person':16,69 'place':111,125,138,147 'proper':11 'receiv':5 'recent':55 'refer':9,45 'singl':22,118 'sixth':98 'speak':30 'stoni':166 'strassburg':156 'stratford':167 'suffic':47 'suffici':27,39,66,121 'us':52 'willow':171 'within':70,75 'without':60 'world':133 'would':46,65",)

This becomes interesting where we want to search for patterns.

For example, let us find those paragraphs where both a phrase similar to "proper names" and a word like "refer" occur. Notice that neither of those occur exactly in paragraph 181, but morphological stems do. Capitalization also will not matter for canonicalized search terms.

In [24]:
query = """
SELECT para_num, raw_text 
FROM books 
WHERE vector @@ to_tsquery('(proper <-> name) & (refer)');
"""
cur.execute(query)
for num, text in cur:
    print(f"{num}: {text[:750]} [...]\n")

181: Just as general names receive a definite concrete reference, so proper
names applicable to different persons come to denote but a single one.
It may be sufficient merely to speak of a man as ‘Charles’ in order to
sufficiently identify him; and indeed such reference would suffice if he
were before us, or had recently been mentioned. Again, even without
this, the name ‘Charles’ would sufficiently identify any person within
his own family, or within any other circle where no other ‘Charles’ was
known. Under other circumstances, we must naturally define him more
closely; as, ‘Charles the Sixth of France,’ ‘Charles the First of
England.’ Just so, there are many places bearing the same name; but a
single name is sufficient to define the place for [...]

1029: The word may be borrowed several times at different periods. It appears
in different forms, of which the more recent bears the stamp of the
parent language, while the older has been exposed to phonetic changes
which have more or le

## Other datatypes

A number of other data types are available for specialized queries.  There are several types for network addresses: CIDR, INET, MACADDR, MACADDR8.  These are string-like, but have custom operators to search subnets.  The UUID type is similarly specialized. 

The XML data type and associated XMLPARSE and XMLSERIALIZE functions are useful for validation of documents, but do not provide any custom operators.  JSON and JSONB are very useful because they can be indexed, and allow queries using the JSONPATH language.  However, explaining that language is outside the scope of this lesson.

Not addressed in this lesson, but available in PostgreSQL, are array, range and composite types.  Arrays can be useful to hold multi-dimensional collections of values.  Ranges can provide validation of values with bounds.  Composite types are occasionally useful, but are very similar simply to creating multiple colums with corresponding types.  For example (from PostgreSQL docs):

```sql
CREATE TYPE inventory_item AS (
    name            text,
    supplier_id     integer,
    price           numeric
);
CREATE TABLE on_hand (
    item      inventory_item,
    count     integer
);

INSERT INTO on_hand VALUES (ROW('fuzzy dice', 42, 1.99), 1000);
```